## Growth media

One essential part of metabolic modeling is applying an environment that mimicks *in vivo* conditions. In metabolic models that is achieved by controlling which metabolites the the model can consume. For instance if we ran an experiment with *E. coli* in a growth medium without Histidine our model should not be able to import Histidine either.

However, metabolites are not really variables in a metabolic model so there is no way of telling the model "there is no Histidine in the growth medium". As an alternative we can apply constraints to the respective import reactions, thus telling the model "there is way you can import Histidine". This can be achieved easily by fixing the respective import reaction at zero flux.

So in summary, the notion COBRApy has of a growth medium is a bit different. Whereas you might be used to see a growth medium as a collection of metabolites and their concentrations COBRApy sees a growth medium **as a collection of import reactions and their maximum import fluxes**. Let's look at a small example to see how that works.  

## Getting and assigning growth media

We will use a small model of *E. coli* central carbon metabolism which is included in COBRApy

In [1]:
from cobra.test import create_test_model

model = create_test_model("textbook")
print("Growth rate =", model.slim_optimize())
model.medium

Growth rate = 0.8739215069684307


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

As we can see by default the model permits imports for most major components. Now, let us simulate anaerobic growth. For that we will make a copy of the original medium and prohibit the import of Oxygen.

In [2]:
anaerobic = model.medium
anaerobic["EX_o2_e"] = 0

You can simply assign the new medium to the model's `medium` attribute. Import reaction that do not appear in the medium dictionary are assumed to be deactivated (fixed at zero flux).

In [3]:
model.medium = anaerobic
print("Growth rate =", model.slim_optimize())
model.medium

Growth rate = 0.21166294973531286


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

As we can see Oxygen does not appear in the medium anymore (missing reactions have an import of zero), however the model still exhibits growth since *E. coli* can grow under anaerobic conditions (albeit with a lower growth rate).

In practice you might want to apply a ceratin growth medium only for one simulation and go back to the previous growth medium afterwards. To make this easy `model.medium` automatically integrates with the COBRApy context manager.

In [4]:
model = create_test_model("textbook")

with model:
    model.medium = anaerobic
    print("Temporary medium =", model.medium)

print("Model medium =", model.medium)

Temporary medium = {'EX_co2_e': 1000.0, 'EX_glc__D_e': 10.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_pi_e': 1000.0}
Model medium = {'EX_co2_e': 1000.0, 'EX_glc__D_e': 10.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_pi_e': 1000.0}


As you can see the anaerobic medium was only applied inside the `with` block.

## Minimal media

Sometimes you might be interested which is the minimal medium that still allows for a certain growth rate. A function for this question (and some others) can be found in the `media` module.

In [5]:
from cobra.media import minimal_medium

minimal_medium(model, min_growth=0.5)

EX_glc__D_e     7.327496
EX_nh4_e        2.726400
EX_o2_e        10.855375
EX_pi_e         1.839350
dtype: float64

By default `minimal_medium` minimizes the total import flux (sum of all import fluxes), but you can also request the minimal medium with the least components. This is the fastest method which also gives nearly unqiue solutions.

In [6]:
minimal_medium(model, min_growth=0.5, minimize_components=True)

EX_glc__D_e    10.000000
EX_nh4_e        3.907885
EX_o2_e        22.707651
EX_pi_e         2.636432
dtype: float64

Note that minimizing the number of components there are many alternative solutions having the same components but different import fluxes. In fact in this case there are several solutions using four different components. To see more than just one possible solution you can pass a number to the `minimize components` argument (the maximum number of alternative solutions). All alternative solutions will have the same number of components but will differ by at least one component (import flux).

In order to observe those alternative solutions in that model we will also have to open previously closed exchnage reactions which can be achieved with the `open_exchanges` argument.

In [7]:
minimal_medium(model, min_growth=0.8, minimize_components=4, open_exchanges=True)

,0,1,2,3
EX_fru_e,0.000000,521.357767,0.000000,0.000000
EX_glc__D_e,0.000000,0.000000,0.000000,519.750758
EX_gln__L_e,0.000000,40.698058,18.848678,0.000000
EX_glu__L_e,348.101944,0.000000,0.000000,0.000000
EX_mal__L_e,0.000000,0.000000,1000.000000,0.000000
EX_nh4_e,0.000000,0.000000,0.000000,81.026921
EX_o2_e,500.000000,0.000000,0.000000,0.000000
EX_pi_e,66.431529,54.913419,12.583458,54.664344


The output will be a pandas DataFrame. This way we can see the many different growth media the model supports. For instance there is only one aerobic medium (#1) and one that uses Fructose instead of Glucose (#2).